## **GF Project 5**

**Project Goal and Objectives**: Have the bot every 30 mins from 9am - 9pm EST give an update on the traffic flow and compare those values with MAs.

### **Bot**:
 - 1) Trigger the file request using a trigger: Trigger + Playwirght 
 - 2) Call the notebook (NEF_Project_5_local.ipynb)
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_%_local.ipynb**:
- 1) Will start by impoting the file (Traffic View 3)
- 2) Calculate all the present stats 
- 3) Insert into the databse the values for the given day and hour 
- 4) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 

#### STEP 1 - Importing Traffic View for a given instance 

In [12]:
import os 
import pandas as pd

current_dir = os.getcwd()
files = os.listdir(current_dir)
for file in files:
    if "Project5" in file and file.endswith('.csv'):
        print(file)
        file_name = file
        
tview1 = pd.read_csv(file_name)
tview1.head()



Project5.csv


,Item,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,Successful Unique Cycle Attempts,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,174,155,89.08%,19,10.92%,163,144,88.34%,19,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$52.85,14.68%,$307.20,85.32%
1,DP1 V3,798,528,66.17%,270,33.83%,494,224,45.34%,270,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$332.40,9.55%,"$3,070.56",88.23%
2,DP1 V3(PP),90,67,74.44%,23,25.56%,90,67,74.44%,23,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$32.72,11.04%,$250.86,84.62%
3,DP12 RP V3,2,2,100.00%,0,0.00%,2,2,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.40,0.00%,$-0.40,0.00%
4,DP12 V3,171,136,79.53%,35,20.47%,145,110,75.86%,35,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$180.48,5.21%,"$3,181.78",91.93%


In [13]:
tview1["Item"].unique()

array(['DP1 RP V3', 'DP1 V3', 'DP1 V3(PP)', 'DP12 RP V3', 'DP12 V3',
       'DP12 V3(PP)', 'DP13 RP V3', 'DP13 V3', 'DP2 V3', 'DP2 V3(PP)',
       'DP22 V3', 'DP22 V3(PP)', 'DP23 V3'], dtype=object)

#### **STEP 2 - Calculate the current statistics**

**Guide**:
- S1 Initials is DP1 from Traffic/Scrub/Prepaid
- S1 RP is DP1RP from Traffic/Scrub/Prepaid
- S1LTV and UPLTV are always static

Main difference here is that we dont have the campaign column, so we need to base our dfs solely from the Item column:
- PP: Prepaid
- RP: Reprocess

In [25]:
# We will start off by calculating S1
main_dict = {}
s1_df = tview1[
      (tview1["Item"] == "DP1 V3(PP)") | (tview1["Item"] == "DP1 V3") | (tview1["Item"] == "DP1 RP V3")
  ]

# Check if column is string/object type before applying string operations
if s1_df["Successful Charges"].dtype == 'object':
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
else:
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


total_s1 = s1_df["Successful Charges"].sum()
print(f"Total S1 Successful Charges: {total_s1}")

# Get the Gross
S1G = total_s1 * 9.49

# Get the new S1
S1N = S1G * 0.3

main_dict["S1"] = [total_s1, S1G, S1N]

Total S1 Successful Charges: 312


C:\Users\mmsou\AppData\Local\Temp\ipykernel_31428\3591349444.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


In [26]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


888.264
2.847


In [4]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


1961.583
2.847


In [28]:
# Now we will calculate S1RP
s1rp_df = tview1[
    (tview1["Item"] == "DP1 RP V3")

]
s1rp_df.head()
s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 19


C:\Users\mmsou\AppData\Local\Temp\ipykernel_31428\172734612.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [30]:
# Now we will calculate UP

up_df = tview1[
      (tview1["Item"] == "DP2 V3") | (tview1["Item"] == "DP2 V3(PP)")
  ]
up_df.head()
up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

total_up = up_df["Successful Charges"].sum()
print(f"Total UP Successful Charges: {total_up:,}")

# Get the Gross
UPG = total_up * 38.11

# Get the Net
UPN = UPG * 0.23

main_dict["UP"] = [total_up, UPG, UPN]


Total UP Successful Charges: 28


C:\Users\mmsou\AppData\Local\Temp\ipykernel_31428\3688151986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [31]:
# Now we will add the WHC row
whc = total_s1 
whc_g = whc * 3
whc_n = whc_g

# Now we add it to the dic 
main_dict["WHC"] = [whc, whc_g, whc_n]


In [32]:
# NOw we will add UR
ur = total_s1
ur_g = ur * 2
ur_n = ur_g

# Now we add it to the dic 
main_dict["UR"] = [ur, ur_g, ur_n]





In [33]:
# Now ew will add S1 RB Proj
s1rb_proj = (total_s1 + total_s1rp) * 0.2
s1rb_proj_g = s1rb_proj * 115.73
s1rb_proj_n = s1rb_proj_g * 0.25

# Now we add it to the dic 
main_dict["S1RB Proj"] = [s1rb_proj, s1rb_proj_g, s1rb_proj_n]


In [34]:
# Now we will ad UP RB Proj
uprb_proj = total_up * 0.25
uprb_proj_g = uprb_proj * 38.11
uprb_proj_n = uprb_proj_g * 0.5

# Now we add it to the dic 
main_dict["UPRB Proj"] = [uprb_proj, uprb_proj_g, uprb_proj_n]




In [35]:
# Now we will calculate S1RP
s1rp_df = tview1[
    (tview1["Item"] == "DP1 RP V3")
]
s1rp_df.head()

s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 19


C:\Users\mmsou\AppData\Local\Temp\ipykernel_31428\4091856167.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [36]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


4784.7029
15.335586217948718


In [38]:
ecpa_df = tview1[
    (tview1["Item"] == "DP1 V3")
]
ecpa_df.head()

ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

dp1_traffic = ecpa_df["Successful Charges"].sum()

ecpa = round((dp1_traffic / ((total_s1 + total_s1rp))) * -30, 2)
print(ecpa)


-24.47


C:\Users\mmsou\AppData\Local\Temp\ipykernel_31428\620824186.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [39]:
S1LTV = 3.50
UPLTV = 1.5




gross_total = 0
for key, values in main_dict.items():
    gross_total += values[1]
print(gross_total)

middle = main_dict["WHC"][1] + main_dict["UR"][1]

RES = (((gross_total - middle)*0.1)/total_s1)+1
print(RES)


NRC = NRCPA + ecpa + S1LTV + UPLTV + RES
print(NRC)







upsell_take = round((total_up / (total_s1))*100, 2)
print(upsell_take)






13868.506000000001
4.945033974358974
0.8106201923076934
8.97


In [40]:
# And in S1ltv
s1ltv = total_s1
s1ltv_g = s1ltv * 4
s1ltv_n = s1ltv_g 

# Add to the dict 
main_dict["S1LTV"] = [s1ltv, s1ltv_g, s1ltv_n]


In [41]:
# And upltv
upltv = total_s1
upltv_g = upltv * 2
upltv_n = upltv_g 

# Add to the dict 
main_dict["UPLTV"] = [upltv, upltv_g, upltv_n]




In [42]:
# Add iin the res 
res = total_s1
res_g = (((((main_dict["S1"][1] + main_dict["S1RP"][1] + main_dict["UP"][1] + main_dict["S1RB Proj"][1] + main_dict["UPRB Proj"][1]) * 0.1) / total_s1) + 3) * total_s1)
res_n = res_g 

# Add to the dict 
main_dict["RES"] = [res, res_g, res_n]




NRC =  ecpa 
print(NRC)


-24.47


In [43]:
# Now we need to add in eCPA
eCPA =  dp1_traffic 
eCPA_g = eCPA * -30
eCPA_n = eCPA_g 

# Add to the dict 
main_dict["eCPA"] = [eCPA, eCPA_g, eCPA_n]  


sum_for_nrc_new = 0
for key, values in main_dict.items():
    if key != "eCPA":
        sum_for_nrc_new += values[2]

nrc_new = sum_for_nrc_new / (total_s1 + total_s1rp)
nrc_new



26.657261329305136

In [44]:
main_dict

{'S1': [312, 2960.88, 888.264],
 'S1RP': [19, 352.45, 42.294],
 'UP': [28, 1067.08, 245.42839999999998],
 'WHC': [312, 936, 936],
 'UR': [312, 624, 624],
 'S1RB Proj': [66.2, 7661.326000000001, 1915.3315000000002],
 'UPRB Proj': [7.0, 266.77, 133.385],
 'S1LTV': [312, 1248, 1248],
 'UPLTV': [312, 624, 624],
 'RES': [312, 2166.8506, 2166.8506],
 'eCPA': [270, -8100, -8100]}

In [45]:
# TV analysis message 
message = "📺 TV message:\n"

# Add teh number of customer, which is total_s1 + total_s1rp
message += f"• Number of customers: {total_s1 + total_s1rp:,}\n"

# Add the upsell take, 
message += f"• Upsell take: {upsell_take:,}%\n"

# Add the ecpa
message += f"• eCPA: {ecpa:,}\n"

# Add the NRC
nrc = nrc_new + ecpa
message += f"• NR/C: {round(nrc, 2):,}\n"


print(message)
list_current = []
list_current.append(total_s1 + total_s1rp)
list_current.append(upsell_take)
list_current.append(ecpa)
list_current.append(nrc)



present_value_customers = (total_s1 + total_s1rp)
present_values_upsell_take = upsell_take
present_value_ecpa = ecpa
present_value_nrc = nrc 

📺 TV message:
• Number of customers: 331
• Upsell take: 8.97%
• eCPA: -24.47
• NR/C: 2.19



### Import the Dataframe 

In [47]:
database_df = pd.read_excel("tv_database_GF.xlsx")

# Create a column where wonly have the date and no time 
database_df["Date"] = database_df["Date-Time"].dt.date

database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP1 V3,DP1 RP V3,total_up
0,2025-06-07 22:00:00,0,0,0,0,2025-06-07,0,0,0,0
1,2025-06-07 22:30:00,0,0,0,0,2025-06-07,0,0,0,0
2,2025-06-07 23:00:00,0,0,0,0,2025-06-07,0,0,0,0
3,2025-06-07 23:30:00,0,0,0,0,2025-06-07,0,0,0,0
4,2025-06-08 00:00:00,0,0,0,0,2025-06-08,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-21 20:30:00,0,0,0,0,2025-06-21,0,0,0,0
438,2025-06-21 21:00:00,0,0,0,0,2025-06-21,0,0,0,0
439,2025-06-21 21:30:00,0,0,0,0,2025-06-21,0,0,0,0
440,2025-06-21 22:00:00,0,0,0,0,2025-06-21,0,0,0,0


In [48]:
# Get the average of the customers, upsell, ecpa, nrc
real_average = database_df["customers"].mean()
real_average_upsell = database_df["upsell"].mean()
real_average_ecpa = database_df["excpa"].mean()
real_average_nrc = database_df["nrc"].mean()



print(real_average)
print(real_average_upsell)
print(real_average_ecpa)
print(real_average_nrc)





0.0
0.0
0.0
0.0


In [49]:
# Get the rounded hour / half hour interval 
from datetime import datetime, timedelta

def get_current_rounded_time(x):
    """
    Get current time rounded to nearest half hour.
    Returns datetime object rounded to nearest half hour.
    """
    now = x
    
    # Calculate minutes to add/subtract to round to nearest half hour
    minutes = now.minute
    if minutes < 15:
        # Round down to previous half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0)
    elif minutes < 45:
        # Round to current half hour
        rounded_time = now.replace(minute=30, second=0, microsecond=0)
    else:
        # Round up to next half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    
    return rounded_time
timey = datetime.now()
datetimee = get_current_rounded_time(timey)
print(datetimee)
database_df.shape

2025-06-23 20:30:00


(442, 10)

In [50]:
from datetime import datetime
from zoneinfo import ZoneInfo
timeest = timey = datetime.now(ZoneInfo("America/New_York"))
timei = get_current_rounded_time(timeest)
timei.hour, timei.minute
timei = timei.strftime("%H:%M")
timei


'15:30'

### In this section we integartes the Attempts for DP1, DP1 RP, DP 2 etc

In [51]:
tview1

,Item,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,Successful Unique Cycle Attempts,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,174,155,89.08%,19,10.92%,163,144,88.34%,19,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$52.85,14.68%,$307.20,85.32%
1,DP1 V3,798,528,66.17%,270,33.83%,494,224,45.34%,270,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$332.40,9.55%,"$3,070.56",88.23%
2,DP1 V3(PP),90,67,74.44%,23,25.56%,90,67,74.44%,23,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$32.72,11.04%,$250.86,84.62%
3,DP12 RP V3,2,2,100.00%,0,0.00%,2,2,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.40,0.00%,$-0.40,0.00%
4,DP12 V3,171,136,79.53%,35,20.47%,145,110,75.86%,35,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$180.48,5.21%,"$3,181.78",91.93%
5,DP12 V3(PP),6,4,66.67%,2,33.33%,6,4,66.67%,2,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$11.08,5.60%,$186.70,94.40%
6,DP13 RP V3,11,11,100.00%,0,0.00%,8,8,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$1.00,0.00%,$-1.00,0.00%
7,DP13 V3,44,36,81.82%,8,18.18%,39,31,79.49%,8,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$33.04,4.18%,$758.08,95.82%
8,DP2 V3,97,70,72.16%,27,27.84%,97,70,72.16%,27,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$83.93,6.50%,"$1,207.75",93.50%
9,DP2 V3(PP),20,19,95.00%,1,5.00%,20,19,95.00%,1,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$6.39,13.36%,$41.45,86.64%


In [ ]:
### In this section we integrate the Attempts for DP1, DP1 RP, DP2 etc

# DP 1
dddp1_df = tview1[tview1["Item"] == 'DP 1']

dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(str)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(float)
averagedp1 = dddp1_df["Attempted Charges"].sum()
print(averagedp1)

# DP 1 RP
dddp1rp_df = tview1[tview1["Item"] == 'DP 1 RP']
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(str)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(float)
averagedp1rp = dddp1rp_df["Attempted Charges"].sum()
print(averagedp1rp)

# DP 2
dddp2_df = tview1[tview1["Item"] == 'DP 2']
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(str)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(float)
averagedp2 = dddp2_df["Attempted Charges"].sum()
print(averagedp2)

# Append these to the list_current
list_current.append(averagedp1)
list_current.append(averagedp1rp)
list_current.append(averagedp2)


KeyError: 'Campaign'

In [27]:
database_df.columns

Index(['Date-Time', 'customers', 'upsell', 'excpa', 'nrc', 'Date', 'Time',
       'DP 1', 'DP 1 RP', 'DP 2'],
      dtype='object')

In [28]:
# Create a new row as a dictionary
new_row = {
    'Date-Time': datetimee,
    'customers': present_value_customers,
    'upsell': present_values_upsell_take,
    'excpa': present_value_ecpa,
    'nrc': present_value_nrc,
    'Date': datetimee.now().date(),
    "Time": datetimee.now().time(),
    "DP 1": averagedp1,
    "DP 1 RP": averagedp1rp,
    "DP 2": averagedp2

}

# Add the row using loc
database_df.loc[len(database_df)] = new_row

# Now we drop the oldest one
database_df.drop(index=0, inplace=True)
database_df = database_df.reset_index(drop=True)
database_df["Time"] = database_df["Date-Time"].dt.time


database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP 1,DP 1 RP,DP 2
0,2025-06-04 22:00:00,1064,44.22,-12.04,18.585741,2025-06-04,22:00:00,2448.0,819.0,1226.0
1,2025-06-04 22:30:00,1072,44.64,-12.03,18.624457,2025-06-04,22:30:00,2462.0,833.0,1279.0
2,2025-06-04 23:00:00,1080,45.33,-12.06,18.686259,2025-06-04,23:00:00,2488.0,848.0,1328.0
3,2025-06-04 23:30:00,1092,45.43,-12.03,18.709051,2025-06-04,23:30:00,2509.0,865.0,1379.0
4,2025-06-05 00:00:00,1104,45.81,-12.09,18.704132,2025-06-05,00:00:00,2532.0,881.0,1433.0
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-17 03:30:00,1446,43.80,-22.76,8.460419,2025-06-17,03:30:00,3649.0,962.0,2014.0
438,2025-06-17 03:00:00,1446,43.80,-22.76,8.460419,2025-06-17,03:00:00,3649.0,962.0,2014.0
439,2025-06-17 03:30:00,1564,43.69,-22.92,8.387123,2025-06-17,03:30:00,3971.0,1017.0,2151.0
440,2025-06-17 18:00:00,768,32.80,-23.16,6.709350,2025-06-17,18:00:00,1872.0,446.0,670.0


##### Get the average values for each half an hour interval

In [29]:
from datetime import time
unique_times = database_df["Time"].unique().tolist()
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

# Create the percenatge change function 
def percentage_change(new, old):
    if new != old:
        p = ((new - old)/old)*100
        return p
    if new == 0 or old == 0:
        return "Invalid"
    else:
        return 1

def alert_maker(list_of_mass, list_of_currentt):
    name_of_componets = ["Customer Sales", "Upsell", "Ecpa", "Nrc", "DP 1", "DP 1 RP", "DP 2"]
    alerts = {}
    alerts_cons = {}
    if not len(list_current) == len(list_mas):
        return None, None
    
    for i in range(len(list_current)):
        new = list_of_currentt[i]
        old = list_of_mass[i]
        diff = percentage_change(new, old)
        diff = round(diff, 2)

        if abs(diff) > 20:
            alerts[name_of_componets[i]] = [new, diff]
        if abs(diff) == 1:
            alerts_cons[name_of_componets[i]] = [new, diff]

    # Return empty dicts instead of None if no alerts
    if not alerts:
        alerts = {}
    if not alerts_cons:
        alerts_cons = {}
        
    print(alerts_cons)
    exclude_keys = {"DP 1", "DP 1 RP", "DP 2"}
    alerts = {k: v for k, v in alerts.items() if k not in exclude_keys}
    return alerts, alerts_cons




print(datetimee.hour, datetimee.minute)
df = database_df[database_df["Time"] == time(datetimee.hour, datetimee.minute)]


# Create a list with all the moving averages
list_mas = []

moving_average_customers = df["customers"].mean()
list_mas.append(moving_average_customers)
moving_average_upsell = df["upsell"].mean()
list_mas.append(moving_average_upsell)
moving_average_ecpa = df["excpa"].mean()
list_mas.append(moving_average_ecpa)
moving_average_ncr = df["nrc"].mean()
list_mas.append(moving_average_ncr)

# Get the previous attemot values for the varipus DPs
df_prev = database_df.iloc[[-2]]
moving_averagedp1 = df_prev["DP 1"].values[0]
list_mas.append(moving_averagedp1)

moving_averagedp1rp = df_prev["DP 1 RP"].values[0]
list_mas.append(moving_averagedp1rp)

moving_averagedp2 = df_prev["DP 2"].values[0]
list_mas.append(moving_averagedp2)

print(f"The values for dp1, dp1rp and dp2 are {moving_averagedp1}, {moving_averagedp1rp}, {moving_averagedp2}")
print()


alert_component = alert_maker(list_mas, list_current)
print(f"The current values for the moving averages are: {list_mas}")
print(f"The current values of this analysis are: {list_current}")
print(alert_component[0])
print(alert_component[1])

18233.694900000002
18 0
The values for dp1, dp1rp and dp2 are 1872.0, 446.0, 670.0

{'DP 1': [1872.0, 1], 'DP 1 RP': [446.0, 1], 'DP 2': [670.0, 1]}
The current values for the moving averages are: [597.0, 34.80571428571429, -18.571428571428573, 10.819937134831152, 1872.0, 446.0, 670.0]
The current values of this analysis are: [768, 32.8, -23.16, 6.709349739583331, 1872.0, 446.0, 670.0]
{'Customer Sales': [768, 28.64], 'Ecpa': [-23.16, 24.71], 'Nrc': [6.709349739583331, -37.99]}
{'DP 1': [1872.0, 1], 'DP 1 RP': [446.0, 1], 'DP 2': [670.0, 1]}


In [30]:
print(alert_component[0])

{'Customer Sales': [768, 28.64], 'Ecpa': [-23.16, 24.71], 'Nrc': [6.709349739583331, -37.99]}


In [31]:
message += "\n"
output_message = message
if alert_component[0] or alert_component[1]:
    if alert_component[0]:
        message += "⚠️We have detected alerts: \n"
        for key, value in alert_component[0].items():
            message += f'• {key} has a deviaition of {value[1]}% from its respective moving average at {timei}. \n'
        output_message = message
    if alert_component[1]:
        for key, value in alert_component[1].items():
            output_message += f'• {key} attempts are equal to those half an hour ago. \n'

print(output_message)


📺 TV message:
• Number of customers: 768
• Upsell take: 32.8%
• eCPA: -23.16
• NR/C: 6.71

⚠️We have detected alerts: 
• Customer Sales has a deviaition of 28.64% from its respective moving average at 13:00. 
• Ecpa has a deviaition of 24.71% from its respective moving average at 13:00. 
• Nrc has a deviaition of -37.99% from its respective moving average at 13:00. 
• DP 1 attempts are equal to those half an hour ago. 
• DP 1 RP attempts are equal to those half an hour ago. 
• DP 2 attempts are equal to those half an hour ago. 



In [32]:
# Delete the file tv_database.xlsx from the directory 
file_path = "tv_database.xlsx"
os.remove(file_path)

# Save the current database_df as tv_database.xlsx
database_df.to_excel("tv_database.xlsx", index=False)